In [49]:
import os
import getpass
import tenacity
import sentinelsat
import ntpath
import telluric as tl

DATA_DIR = '/media/lddm/LDDM/forest_monitoring'
DOWNLOADS_DIR = DATA_DIR + '/archives'
NUMBER_OF_PRODUCTS = 2

In [47]:
def get_products_filenames(downloaded):
    products_filenames = {}
    for product_id, product_info in downloaded.items():
        products_filenames[product_id] = ntpath.basename(product_info['path'])
        
    return products_filenames

### Obtain scihub credentials

In [20]:
if not ('SCIHUB_USERNAME' in os.environ and
        'SCIHUB_PASSWORD' in os.environ):
    os.environ['SCIHUB_USERNAME'] = input('Scihub Username:')
    os.environ['SCIHUB_PASSWORD'] = getpass.getpass('Scihub Password:')

In [27]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

api = SentinelAPI(os.environ['SCIHUB_USERNAME'], os.environ['SCIHUB_PASSWORD'],
                  'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('ROIs/uruguay_forest.geojson'))
products = api.query(footprint,
                     date=('NOW-14DAYS', 'NOW'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30),
                     producttype='S2MSI2A')

In [28]:
products_id_to_download = list(products.keys())[:2]
products_id_to_download

['9c66e7ed-acb8-4a1d-bee9-b0686254e154',
 '28a35512-af84-456f-bf8e-335e59d12e3a']

In [29]:
# download all results from the search
@tenacity.retry(stop=tenacity.stop_after_attempt(3), wait=tenacity.wait_fixed(3600))
def download_all(*args, **kwargs):
    return api.download_all(*args, **kwargs)

downloaded, triggered, failed = download_all(products_id_to_download, directory_path=DOWNLOADS_DIR)

# print('Downloaded {}'.format(downloaded))
# print('Triggered {}'.format(triggered))
# print('Failed {}'.format(failed))

In [37]:
print(downloaded)

{'9c66e7ed-acb8-4a1d-bee9-b0686254e154': {'id': '9c66e7ed-acb8-4a1d-bee9-b0686254e154', 'title': 'S2B_MSIL2A_20210422T133219_N0300_R081_T21HXE_20210422T154725', 'size': 80935060, 'md5': 'ccb3a0a713bbcfa492b9a0984ec4472d', 'date': datetime.datetime(2021, 4, 22, 13, 32, 19, 24000), 'footprint': 'POLYGON((-54.96396 -32.608302309198976,-54.956207 -32.58273095098718,-54.911865 -32.435942372453425,-54.867584 -32.28913852765143,-54.823547 -32.142223277939145,-54.77957 -31.995315974316185,-54.779114 -31.993716684108204,-54.76425 -32.605676852027074,-54.96396 -32.608302309198976))', 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('9c66e7ed-acb8-4a1d-bee9-b0686254e154')/$value", 'Online': True, 'Creation Date': datetime.datetime(2021, 4, 24, 13, 43, 39, 291000), 'Ingestion Date': datetime.datetime(2021, 4, 22, 21, 36, 50, 239000), 'path': '/media/lddm/LDDM/forest_monitoring/archives/S2B_MSIL2A_20210422T133219_N0300_R081_T21HXE_20210422T154725.zip', 'downloaded_bytes': 0}, '28a35512-a

In [41]:
products_filenames = get_products_filenames(downloaded)
products_filenames

{'9c66e7ed-acb8-4a1d-bee9-b0686254e154': 'S2B_MSIL2A_20210422T133219_N0300_R081_T21HXE_20210422T154725.zip',
 '28a35512-af84-456f-bf8e-335e59d12e3a': 'S2A_MSIL2A_20210420T134211_N0300_R124_T21HXE_20210420T173838.zip'}

### Automatically obtain `True Color Image (TCI)` from downloaded package and save it as a GeoTiff

In [48]:
import calval
from calval.providers.sentinel import SceneInfo, SceneData

# scene_id = next(iter(products)) 
# filenames = products[scene_id]['filename'].replace('.SAFE', '.zip')
# print(filenames)
# print('Uploading file : ', filenames)
scenes_info = {}
scenes_data = {}
for product_id, product_filename in products_filenames.items():
    config = {
        'data_dir': DATA_DIR
    }
    print(product_filename)
    scene_info = SceneInfo.from_filename(filename=product_filename, config=config)
    scene_info.extract_archive()
    scenes_data[product_id] = SceneData.from_sceneinfo(scene_info)
    scenes_info[product_id] = scene_info

S2B_MSIL2A_20210422T133219_N0300_R081_T21HXE_20210422T154725.zip
S2A_MSIL2A_20210420T134211_N0300_R124_T21HXE_20210420T173838.zip


/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/calval/providers/scene_info.py:29: UserWarning: /media/lddm/LDDM/forest_monitoring/scenes/S2B_MSIL2A_20210422T133219_N0300_R081_T21HXE_20210422T154725.SAFE already exists, not extracting
  warnings.warn('{} already exists, not extracting'.format(output_path))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/calval/providers/scene_info.py:29: UserWarning: /media/lddm/LDDM/forest_monitoring/scenes/S2A_MSIL2A_20210420T134211_N0300_R124_T21HXE_20210420T173838.SAFE already exists, not extracting
  warnings.warn('{} already exists, not extracting'.format(output_path))


In [ ]:
### Automatically obtain `True Color Image (TCI)` from downloaded package and save it as a GeoTiff


In [51]:
for product_id, scene_info in scenes_info.items():
    tci_path = scene_info.get_band_path('TCI')
    geo_raster = tl.GeoRaster2.open(tci_path)
    geo_raster.save(product_id + '.tif')

In [ ]:
geo_raster

In [ ]:
geo_raster.save('test_s2.tiff')